In [22]:
import os
import numpy as np
import subprocess

In [23]:
rnk_path1 = '../gordo_retrieval/finetuned_best'
rnk_path2 = '../gordo_retrieval/gordo_multires'
eval_file = '../gordo_retrieval/datasets/evaluation/compute_ap'
lab_root = '../gordo_retrieval/datasets/CLICIDE/lab'

In [24]:
def f_2_name(fname):
    return fname.split('/')[-1].split('.')[0]
def f_2_lab(fname):
    return fname.split('/')[-1].split('-')[0]

In [25]:
clicide_qnames = [f_2_name(fname)[:-len('_query')] for fname in os.listdir(lab_root) if fname.endswith('_query.txt')]

In [26]:
def ap_rnk(rnk_file):
    cmd = "{0} {1} {2}".format(eval_file, os.path.join(lab_root, f_2_name(rnk_file)), rnk_file)
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    map_ = float(p.stdout.readlines()[0])
    p.wait()
    return map_

def prec1_rnk(rnk_file):
    with open(rnk_file) as f:
        res = int(f.readline().split('-')[0] == f_2_lab(rnk_file))
    return res

In [27]:
results = {}
ap_threshold = 0.2
for rnk_path in (rnk_path1, rnk_path2):
    results[rnk_path] = {}
    for rnk_file in os.listdir(rnk_path):
        name = f_2_name(rnk_file)
        if name not in clicide_qnames:
            continue

        rnk_file = os.path.join(rnk_path, rnk_file)
        ap = ap_rnk(rnk_file)
        prec1 = prec1_rnk(rnk_file)
        results[rnk_path][name] = (ap, prec1, ap < ap_threshold)
    print('Analysis of: {0}'.format(rnk_path))
    print('mean AP: {0}'.format(np.mean([results[rnk_path][name][0] for name in results[rnk_path]])))
    print('mean Prec1: {0}'.format(np.mean([results[rnk_path][name][1] for name in results[rnk_path]])))
    print('#files with AP<{0}: {1}'.format(ap_threshold, sum(results[rnk_path][name][2] for name in results[rnk_path])))
    print('mean AP of these files: {0}'.format(np.mean([results[rnk_path][name][0] for name in results[rnk_path] if results[rnk_path][name][2]])))

Analysis of: ../gordo_retrieval/finetuned_best
mean AP: 0.74013620863
mean Prec1: 0.787878787879
#files with AP<0.2: 29
mean AP of these files: 0.0479975663448
Analysis of: ../gordo_retrieval/gordo_multires
mean AP: 0.703579815697
mean Prec1: 0.927272727273
#files with AP<0.2: 18
mean AP of these files: 0.0794265327778


In [28]:
print('Difficult images for both')
for name in results[rnk_path1]:
    if results[rnk_path1][name][2] and results[rnk_path2][name][2]:
        print(name, results[rnk_path1][name], results[rnk_path2][name])
print('Difficult images for us, not them')
for name in results[rnk_path1]:
    if results[rnk_path1][name][2] and not results[rnk_path2][name][2]:
        print(name, results[rnk_path1][name], results[rnk_path2][name])

Difficult images for both
('1F-0452', (0.00817937, 0, True), (0.107334, 1, True))
('1C-0454', (0.0137451, 0, True), (0.112644, 0, True))
('4L-0504', (0.000320924, 0, True), (0.00165017, 0, True))
('12G-0428', (0.122469, 0, True), (0.156106, 1, True))
('5P-0508', (0.0494926, 0, True), (0.0244491, 0, True))
('8F-0517', (0.0837345, 0, True), (0.119059, 1, True))
('2J-0458', (0.0910156, 0, True), (0.137268, 1, True))
('5B-0506', (0.00202822, 0, True), (0.0132368, 0, True))
('2A-0456', (0.0735675, 0, True), (0.16275, 1, True))
('16C-0343', (0.0152826, 0, True), (0.107037, 1, True))
('23G-0738', (0.0131184, 0, True), (0.142712, 1, True))
('11C-0351', (0.00422175, 0, True), (0.00396822, 0, True))
('2H-0460', (0.0770878, 0, True), (0.145631, 1, True))
('26J-1245', (0.00104124, 0, True), (0.00562519, 0, True))
('2E-0495', (0.145701, 0, True), (0.140106, 1, True))
Difficult images for us, not them
('22D-0742', (0.0106182, 0, True), (0.224588, 1, False))
('23B-0816', (0.0642858, 0, True), (0.2655

In [29]:
print('Easy images for both')
for name in results[rnk_path1]:
    if results[rnk_path1][name][0] >= 1. and results[rnk_path2][name][0] >= 1.:
        print(name, results[rnk_path1][name], results[rnk_path2][name])

Easy images for both
('41C-1223', (1.0, 1, False), (1.0, 1, False))
('41E-1221', (1.0, 1, False), (1.0, 1, False))
('25B-1230', (1.0, 1, False), (1.0, 1, False))
('23D-0740', (1.0, 1, False), (1.0, 1, False))
('44C-1218', (1.0, 1, False), (1.0, 1, False))
('31A-0566', (1.0, 1, False), (1.0, 1, False))
('37K-1216', (1.0, 1, False), (1.0, 1, False))
('26K-1234', (1.0, 1, False), (1.0, 1, False))
('26K-1233', (1.0, 1, False), (1.0, 1, False))
('42E-1220', (1.0, 1, False), (1.0, 1, False))
('11J-0521', (1.0, 1, False), (1.0, 1, False))
('28I-1246', (1.0, 1, False), (1.0, 1, False))
('26P-1232', (1.0, 1, False), (1.0, 1, False))
('20F-0729', (1.0, 1, False), (1.0, 1, False))
('27C-1240', (1.0, 1, False), (1.0, 1, False))
('23K-0717', (1.0, 1, False), (1.0, 1, False))
('31A-1252', (1.0, 1, False), (1.0, 1, False))
('39C-1258', (1.0, 1, False), (1.0, 1, False))
('1L-0489', (1.0, 1, False), (1.0, 1, False))
('41A-1222', (1.0, 1, False), (1.0, 1, False))
('28C-1244', (1.0, 1, False), (1.0, 1, F